# Packages and Initialization

In [1]:
library(tidyverse)
library(httr2) # for making api requests
library(jsonlite) # for making

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: 'jsonlite'


The following object is masked from 'package:purrr':

    flatten




# Loading Data

In [20]:
player_url <- "https://ch.tetr.io/api/users/2hp/records/league/recent"

In [21]:
req <- request(player_url)|>
    req_user_agent("RemiliaScarlet")|>
    req_throttle( rate = 1)
resp <- req |>
    req_perform()
resp_json <- resp |> resp_body_string()

In [43]:
json_data <- fromJSON(resp_json)
json_data_frame_rounds <- as.data.frame(json_data)|>
        flatten(recursive = TRUE)|>
        select(data.entries._id,
                data.entries.results.rounds)|>
        unnest_wider(data.entries.results.rounds, names_sep = "_",
                       names_repair = 'unique')|>
        unnest_wider(data.entries.results.rounds_1:last_col(),
                     names_sep = "_",
                     names_repair = 'unique'
                    )|>
        select(!(contains('shadows')|contains('shadowedBy')))|>
        unnest_wider(contains('stats'),
                     names_sep = "_",
                     names_repair = 'unique'
                    )|>
        unnest_wider(contains('data.entries.results.rounds_'),
                     names_sep = "_",
                     names_repair = 'unique')|>
        select(!(contains('targetingfactor')|
                 contains('targetinggrace')|
                 contains('altitude')|
                 data.entries.results.rounds_1_id_1|
                 data.entries.results.rounds_1_id_2))

In [74]:
json_data <- fromJSON(resp_json)
json_data_frame_matches <- as.data.frame(json_data)|>
        flatten(recursive = TRUE)|>
        select(data.entries._id,
              data.entries.results.leaderboard,
              data.entries.extras.result)|>
        unnest_wider(data.entries.extras.result, names_sep = "_",
                       names_repair = 'unique')|>
        unnest_wider(data.entries.results.leaderboard,
                     names_sep = ".",
                     names_repair = 'unique'
                    )

In [76]:
ls(json_data_frame_matches)

[1] "data.entries._id"                             
 [2] "data.entries.extras.result_1"                 
 [3] "data.entries.results.leaderboard.active"      
 [4] "data.entries.results.leaderboard.id"          
 [5] "data.entries.results.leaderboard.naturalorder"
 [6] "data.entries.results.leaderboard.shadowedBy"  
 [7] "data.entries.results.leaderboard.shadows"     
 [8] "data.entries.results.leaderboard.stats"       
 [9] "data.entries.results.leaderboard.username"    
[10] "data.entries.results.leaderboard.wins"

In [ ]:
ggplot( json_data_frame_matches, aes(x = vsscore_player0, y = vsscore_player1, color = 	data.entries.results.leaderboard_wins.1))+
geom_point()